In [1]:
#pip install keras-nlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 15.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.4 MB/s eta 0:00:0000:01
  Attempting uninstall: keras
    Found existing installation: keras 3.2.1
    Uninstalling keras-3.2.1:
      Successfully uninstalled keras-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from tensorflow import keras
import keras_nlp
import matplotlib.pyplot as plt

2024-05-09 12:25:54.547648: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 12:25:54.547732: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 12:25:54.721397: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
#reading the validation data
df = pd.read_csv('/kaggle/input/memotionnlp/updated_val.csv')

replace_value = {
    'negative': 0,
    'very_negative': 0,
    'neutral': 1,
    'positive': 2,
    'very_positive': 2
}
#iterating over the dataframe if ocr is null drop the frame
for index, row in df.iterrows():
    if pd.isna(row['ocr']):
        df.drop(index, inplace=True)

#defining X_valid and Y_valid
X_val = df['ocr']
Y_val = df['overall']

#changing into numeric values
Y_val = Y_val.replace(replace_value);
Y_val = np.array(Y_val,dtype=np.int32)

#changing into list
X_val = X_val.tolist()
X_val = [s.replace('\n', ' ') for s in X_val]

/tmp/ipykernel_34/3956033500.py:21: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Y_val = Y_val.replace(replace_value);


In [3]:
import json

# Assuming '25epoch_model.json' contains custom objects used in your Keras model
with open('/kaggle/input/modeljson/25epoch_model.json', 'r') as json_file:
    custom_objects = json.load(json_file)

In [ ]:
import shutil

# Specify the source and destination paths
source_path = "/kaggle/input/distilbertclassifiermemotion/keras/./1/25epoch_model.keras"
destination_path = "/kaggle/working/25epoch_model.keras"

# Copy the model file to a writable directory
shutil.copyfile(source_path, destination_path)

# Load the model from the copied file
model = keras.models.load_model(destination_path)



/opt/conda/lib/python3.10/site-packages/keras/src/saving/serialization_lib.py:727: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)


In [6]:
Y_cal = model.predict(x=X_val)

I0000 00:00:1714137018.897344     107 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-04-26 13:10:18.903529: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.


47/47 [==============================] - 160s 3s/step


In [7]:
Y_cal = np.argmax(Y_cal, axis=1)
Y_cal

array([1, 1, 1, ..., 1, 2, 1])

In [8]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Calculate precision
precision = precision_score(Y_val, Y_cal, average='macro')

# Calculate recall
recall = recall_score(Y_val, Y_cal, average='macro')

# Calculate F1 score
f1 = f1_score(Y_val, Y_cal, average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Precision: 0.33533457667518096
Recall: 0.3373394806074341
F1 Score: 0.33394178427747817
